In [ ]:
!pip install -U portalocker>=2.0.0

## PyTorch

## Output stored in csv

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Loading the datasets
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Neural Network Definition
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        return x

# Experiment configurations
configurations = [
    {"layer_sizes": [28*28, 64, 10], "optimizer": "SGD", "lr": 0.01, "batch_size": 64},
    {"layer_sizes": [28*28, 128, 10], "optimizer": "Adam", "lr": 0.001, "batch_size": 50},
    {"layer_sizes": [28*28, 64, 32, 10], "optimizer": "RMSprop", "lr": 0.0005, "batch_size": 100},
    {"layer_sizes": [28*28, 128, 64, 10], "optimizer": "SGD", "lr": 0.01, "batch_size": 32},
    {"layer_sizes": [28*28, 256, 128, 64, 10], "optimizer": "Adam", "lr": 0.001, "batch_size": 128},
    {"layer_sizes": [28*28, 256, 128, 10], "optimizer": "RMSprop", "lr": 0.0001, "batch_size": 75},
    {"layer_sizes": [28*28, 512, 256, 128, 10], "optimizer": "SGD", "lr": 0.005, "batch_size": 60},
    {"layer_sizes": [28*28, 512, 256, 10], "optimizer": "Adam", "lr": 0.001, "batch_size": 45},
    {"layer_sizes": [28*28, 128, 64, 32, 16, 10], "optimizer": "RMSprop", "lr": 0.0005, "batch_size": 30},
    {"layer_sizes": [28*28, 1024, 512, 256, 128, 64, 10], "optimizer": "SGD", "lr": 0.002, "batch_size": 90},
    {"layer_sizes": [28*28, 32, 10], "optimizer": "SGD", "lr": 0.005, "batch_size": 128},
    {"layer_sizes": [28*28, 256, 128, 64, 10], "optimizer": "Adam", "lr": 0.0005, "batch_size": 100},
    {"layer_sizes": [28*28, 64, 10], "optimizer": "RMSprop", "lr": 0.0001, "batch_size": 32},
    {"layer_sizes": [28*28, 512, 256, 10], "optimizer": "SGD", "lr": 0.001, "batch_size": 75},
    {"layer_sizes": [28*28, 512, 256, 10], "optimizer": "Adam", "lr": 0.0005, "batch_size": 45},
    {"layer_sizes": [28*28, 128, 64, 10], "optimizer": "RMSprop", "lr": 0.0001, "batch_size": 60},
    {"layer_sizes": [28*28, 1024, 512, 256, 64, 10], "optimizer": "SGD", "lr": 0.0015, "batch_size": 90},
    {"layer_sizes": [28*28, 64, 32, 10], "optimizer": "Adam", "lr": 0.0005, "batch_size": 50},
    {"layer_sizes": [28*28, 256, 128, 10], "optimizer": "RMSprop", "lr": 0.0001, "batch_size": 75},
    {"layer_sizes": [28*28, 512, 256, 128, 10], "optimizer": "SGD", "lr": 0.001, "batch_size": 60}
]

import csv

# Train and evaluate each configuration
results = []
for idx, config in enumerate(configurations):
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    model = MLP(config["layer_sizes"])

    if config["optimizer"] == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=config["lr"])
    elif config["optimizer"] == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    elif config["optimizer"] == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=config["lr"])

    criterion = nn.CrossEntropyLoss()
    num_epochs = 20  # Adjust the number of epochs for detailed analysis

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Evaluation after each epoch
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total

        # Append results for each epoch
        results.append({
            "Configuration": idx + 1,
            "Layer Sizes": config["layer_sizes"],
            "Optimizer": config["optimizer"],
            "Learning Rate": config["lr"],
            "Batch Size": config["batch_size"],
            "Epoch": epoch + 1,
            "Accuracy": f'{accuracy:.2f}%'
        })

# Write results to a CSV file
with open('results.csv', mode='w', newline='') as file:
    fieldnames = ['Configuration', 'Layer Sizes', 'Optimizer', 'Learning Rate', 'Batch Size', 'Epoch', 'Accuracy']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)